In [1]:
# Load libraries
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.cluster import KMeans
import collections

In [2]:
# Import text data
raw_training = pd.read_csv("training.csv")
raw_testing = pd.read_csv("test.csv")

# Create bag of words
count = CountVectorizer()
bag_of_words = count.fit_transform(raw_training["article_words"])

# Create feature matrix
X = bag_of_words

# Create bag of words
y = raw_training["topic"].apply(hash)

# Create number of clusters
y_clusters = len(raw_training.groupby("topic").count())

# testing y
y_valid = raw_testing["topic"].apply(hash)

#transform testing data
X_valid = count.transform(raw_testing["article_words"])

#######################Resampling Dataset#######################


# Reducing the effect imbalnced by deleting some irrelevant class
# "Irrelevant" classe has 4734 samples in the training data, try to reduce it into 2000
irrelevant = raw_training[raw_training["topic"] == "IRRELEVANT"]
remove_n = 2734
drop_indices = np.random.choice(irrelevant.index, remove_n, replace=False)
irrelevant = irrelevant.drop(drop_indices)

reduce_training =  pd.concat([raw_training[raw_training["topic"] != "IRRELEVANT"], irrelevant],ignore_index=True)
reduce_bag_of_words = count.fit_transform(reduce_training["article_words"])
R_X = reduce_bag_of_words
R_y = reduce_training["topic"].apply(hash)
# Create number of clusters
R_y_clusters = len(reduce_training.groupby("topic").count())
#transform testing data
R_X_valid = count.transform(raw_testing["article_words"])

# Icreasing the minor classes
# Increasing 
topic_class = raw_training[raw_training["topic"] != "IRRELEVANT"]
increase_training = pd.concat([topic_class, topic_class, topic_class, raw_training[raw_training["topic"] == "IRRELEVANT"]], ignore_index=True)
increase_bag_of_words = count.fit_transform(increase_training["article_words"])
I_X = increase_bag_of_words
I_y = increase_training["topic"].apply(hash)
# Create number of clusters
I_y_clusters = len(increase_training.groupby("topic").count())
#transform testing data
I_X_valid = count.transform(raw_testing["article_words"])

# Considering stop words
count2 = CountVectorizer(stop_words='english')
sw_bag_of_words = count.fit_transform(increase_training["article_words"])
s_X = sw_bag_of_words
s_y = increase_training["topic"].apply(hash)
# Create number of clusters
s_y_clusters = len(increase_training.groupby("topic").count())

#transform testing data
s_X_valid = count.transform(raw_testing["article_words"])

In [3]:
# Using specific model "method", return specifc score "score" by cross validation

def Model_Score(X, y, val_X, val_y, method):
   
    clf = method
    clf.fit(X, y)
    y_predicted = clf.predict(val_X)
    
    accuracy = accuracy_score(val_y, y_predicted)
    precision = precision_score(val_y, y_predicted, average = "macro")
    recall = recall_score(val_y, y_predicted, average = "macro")
    f1 = f1_score(val_y, y_predicted, average = "macro")
    
    return accuracy, precision, recall, f1

**KMeans**

In [4]:
raw_accuracy, raw_precision, raw_recall, raw_f1 = Model_Score(X, y, X_valid, y_valid, KMeans(n_clusters=y_clusters))

# reduce irrelevant samples
R_accuracy, R_precision, R_recall, R_f1 = Model_Score(R_X, R_y, R_X_valid, y_valid, KMeans(n_clusters=R_y_clusters))

# Icreasing the minor classes
I_accuracy, I_precision, I_recall, I_f1 = Model_Score(I_X, I_y, I_X_valid, y_valid, KMeans(n_clusters=I_y_clusters))

# Try considering stop words
s_accuracy, s_precision, s_recall, s_f1 = Model_Score(s_X, s_y, s_X_valid, y_valid, KMeans(n_clusters=s_y_clusters))

C:\Users\anhvu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\anhvu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\anhvu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\anhvu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [5]:
print("====================================================================================")

print("KMeans using raw data:\naccuracy:  " + str(raw_accuracy) +
     "\nprecision: " + str(raw_precision) + "\nrecall:    " + str(raw_recall) + "\nf1:        " +
     str(raw_f1))
print("====================================================================================")

print("KMeans with irrelevant classes:\naccuracy:  " + str(R_accuracy) +
     "\nprecision: " + str(R_precision) + "\nrecall:    " + str(R_recall) + "\nf1:        " +
     str(R_f1))
print("====================================================================================")

print("KMeans with increasing minor classes:\naccuracy:  " + str(I_accuracy) +
     "\nprecision: " + str(I_precision) + "\nrecall:    " + str(I_recall) + "\nf1:        " +
     str(I_f1))
print("====================================================================================")

print("KMeans using minorcalsses with stop words:\naccuracy:  " + str(s_accuracy) +
     "\nprecision: " + str(s_precision) + "\nrecall:    " + str(s_recall) + "\nf1:        " +
     str(s_f1))

KMeans using raw data:
accuracy:  0.0
precision: 0.0
recall:    0.0
f1:        0.0
KMeans with irrelevant classes:
accuracy:  0.0
precision: 0.0
recall:    0.0
f1:        0.0
KMeans with increasing minor classes:
accuracy:  0.0
precision: 0.0
recall:    0.0
f1:        0.0
KMeans using minorcalsses with stop words:
accuracy:  0.0
precision: 0.0
recall:    0.0
f1:        0.0
